## edgeR TMM-Normalization with mRNA counts table

In [ ]:
options(stringsAsFactors = FALSE)

In [ ]:
#Load packages
library(limma)
library(edgeR)
library(data.table)
library(RColorBrewer)
library(gplots)
library(ggplot2)
library(biomaRt)
library(apeglm)
library(DESeq2)

<div class="alert alert-block alert-success">
    <p><b>Your turn!</b> Try to go through the edgeR portion of the notebook using the mouse data. Create the DGE object and generate the plots. Fill out everything up to the section on Limma-voom.  </p>
</div>

### Creating DGE object for edgeR
Read in the count data `data/featCounts_mouse_E18.csv` and metadata `data/mouse_metadata.csv`

In [ ]:
meta <- read.csv("FILE", stringsAsFactors=F)
counts <- read.csv("FILE", stringsAsFactors=F, row.names=1)
head(meta)
head(counts)

Make DGE object: 
- Define `group` using the `Group` column from `meta`
- Define `design` using `group`
- Create `dge` using `DGEList()`

In [ ]:
#define DGE list
#define groups
group<- VARIABLE
group
design <- model.matrix(~0+VARIABLE)
dge<- DGEList(counts=VARIABLE,group=VARIABLE)

Plot library sizes using `dge$samples$lib.size`

In [ ]:
#lib sizes
par(mar=c(10,5,5,5))
options(scipen=5)
barplot(VARIABLE, horiz=F, names.arg=colnames(dge$counts), las=2, cex.name = 0.8, cex.axis=.9, 
        main="Library Size")

In [ ]:
#To check
class(dge)
dim(dge)
dge$samples

<div class="alert alert-block alert-warning">
    The above cell should match these:
</div>

<p><code>class(dge)</code> should return 'DGEList' </p>
<p><code>dim(dge)</code> should return 487956 · 6 </p>
<p><code>dge$samples</code> should return </p>

|                 | group<br><fct> | lib.size<br><dbl> | norm.factors<br><dbl> |
|-----------------|----------------|-------------------|-----------------------|
| E18_Double_KO_1 | KO             | 19967593          | 1                     |
| E18_Double_KO_2 | KO             | 26123103          | 1                     |
| E18_Double_KO_3 | KO             | 22356289          | 1                     |
| E18_WT_2        | WT             | 23864118          | 1                     |
| E18_WT_3        | WT             | 30210498          | 1                     |
| E18_WT_4        | WT             | 21391109          | 1                     |

### Filtering based on cpm cutoff 
Filter miRNAs with at least 10 cpm present in at least 3 samples

In [ ]:
#filter
table(rowSums(dge$counts==0)==6)
keep <- rowSums(cpm(dge)>NUMBER) >= NUMER
dge.f <- dge[keep, , keep.lib.sizes=FALSE]
dim(dge.f)

### Estimate dispersion

Generate the estimate dispersion `d` with `estimateDisp()` using the filtered dge (`dge.f`).

In [ ]:
#estimate dispersions
d <- estimateDisp(VARIABLE, design=VARIABLE)

In [ ]:
d$samples$lib.size
summary(d$prior.df)
sqrt(d$common.disp)

### Normalization using TMM
“TMM (weighted trimmed mean of log expression) determines scaling factor calculated after double trimming values at the two extremes based on log-intensity ratios (M-values) and log-intensity averages (A-values)” (Dillies et al. Briefings in Bioinformatics, Vol. 14 (6): 671–683, 2013)

Calculate
- cpm (cpm)
- log cpm (lcpm)
- filtered log cpm (lcpm.f)
- log cpm normalized (lcpm.norm)

In [ ]:
cpm <- ADD
lcpm <- ADD
dge.norm <- ADD
lcpm.f <- ADD
lcpm.norm <- ADD

## Plots

### Filtered and unfiltered data 
Create the density plots for the raw data (log cpm) and the filtered data (log cpm filtered)

In [ ]:
#set colours for graphs
nsamples<-ncol(dge.norm)
col <- brewer.pal(nsamples, "Paired")

#Visualise filtered vs unfiltered data
par(mfrow=c(1,2))

#plot unfiltered data (log cpm)
samplenames<-meta$Sample_ID

plot(density(VARIABLE[,1]),col=col(lcpm,as.factor = FALSE),lwd=1,ylim=c(0,2),las=2,main="",xlab="")

title(main="A. Raw data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
  den <- density(lcpm[,i])
  lines(den$x, den$y, col=col[i], lwd=2)
}
legend("topright", samplenames, text.col=col, bty="n",cex=0.8,pt.cex=0.8)

#plot filtered data (log cpm filtered)
plot(density(VARIABLE[,1]), col=col(lcpm.f,as.factor=FALSE), lwd=2, ylim=c(0,0.5), las=2,
     main="", xlab="")
title(main="B. Filtered data", xlab="Log-cpm")
abline(v=0, lty=3)
for (i in 2:nsamples){
  den <- density(lcpm.f[,i])
  lines(den$x, den$y, col=col[i], lwd=2)
}
legend("topright", samplenames, text.col=col, bty="n",cex=0.8,pt.cex=0.8)


### Boxplots of TMM-Normalized vs. unnormalized data

In [ ]:
# Unnormalized data (log cpm)
par(mfrow=c(1,2))
dge$samples$norm.factors
boxplot(VARIABLE,las=2, col=col, main="",ylim=c(-10,20),names=meta$Sample_ID,cex=0.2)
title(main="A. Unnormalized data",ylab="Log-cpm")

# Normalized data (log cpm normalized)
dge.norm$samples$norm.factors
boxplot(VARIABLE, las=2, col=col, main="", ylim=c(-10,20),names=meta$Sample_ID,cex=0.2)
title(main="B. TMM Normalized data",ylab="Log-cpm")

### PCA plot
Make a PCA plot for the log cpm normalized data.
- Set `Group` as factors for the data (Hint: use `as.factors()` with the "Group" column in `meta`)
- Use `plotMDS()` to create the PCA plot
    - What should you put for the `labels=` parameter?

In [ ]:
#MDS Plots
Group <- as.factor(meta$Group)
col.1 <- Group
char.1 <- Group
nlevels(col.1)

levels(col.1) <-  brewer.pal(nlevels(col.1), "Paired") #if more than 2 conditions
col.1 <- as.character(col.1)
levels(char.1) <- c(0:nlevels(char.1))

plotMDS(VARIABLE,top = 500, labels=VARIABLE, col=col.1, cex=1, pch=19,xlab="Leading logFC dim1", 
        ylab="Leading LogFC dim 2")
title(main="Group")

## Limma-voom

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4937821/

### Removing heteroscedascity from count data

It has been shown that for RNA-seq count data, the variance is not independent of the mean – this is true of raw counts or when transformed to log-CPM values. Methods that model counts using a Negative Binomial distribution assume a quadratic mean-variance relationship. In limma, linear modelling is carried out on the log-CPM values which are assumed to be normally distributed and the mean-variance relationship is accommodated using precision weights calculated by the voom function.

When operating on a DGEList-object, voom converts raw counts to log-CPM values by automatically extracting library sizes and normalisation factors from x itself. 

Typically, the “voom-plot” shows a decreasing trend between the means and variances resulting from a combination of technical variation in the sequencing experiment and biological variation amongst the replicate samples from different cell populations. Experiments with high biological variation usually result in flatter trends, where variance values plateau at high expression values. Experiments with low biological variation tend to result in sharp decreasing trends.

First set up the `design` matrix.

In [ ]:
#limma-voom 
#Set up design
design <- model.matrix(~0 + group)
colnames(design) <- gsub("group","", colnames(design))
design

Then use [`makeContrasts()`](https://www.rdocumentation.org/packages/limma/versions/3.28.14/topics/makeContrasts) to "express contrasts between a set of parameters as a numeric matrix".

We want to contrast the WT and KO mice, so we need to use the notation `WT-KO` and use the `design` matrix as our levels.

In [ ]:
cm <- makeContrasts(WTvsKO=VARIABLE,levels=VARIABLE)

Apply `voom()` on the normalized dges (`dge.norm`) to remove heteroscedasticity from count data

In [ ]:
v <- voom(VARIABLE, design, plot=TRUE)
write.csv(v$E, "output/TMM_and_Voom_normalized_counts.csv")

Fit the linear model

In [ ]:
vfit <- lmFit(v,design)
vfit <- contrasts.fit(vfit, contrasts=cm)
efit <- eBayes(vfit)
plotSA(efit, main="Final model: Mean-variance trend")

Use `decideTests()` on `efit` to determine which genes are up-regulated, down-regulated or not significantly different.

In [ ]:
dt <- decideTests(VARIABLE)
summary(dt)

### mRNAs with p<0.01

In [ ]:
#write out p0.01 RNAs
WT_vs_KO<-topTreat(efit,coef=1,n=Inf)
head(WT_vs_KO)
ENSID<-row.names(WT_vs_KO)
norm<-data.frame(v$E)
merged<-merge(WT_vs_KO,norm,by=0,all=TRUE)
final<-subset(merged,merged$adj.P.Val<0.01)
write.table(final,file="output/final_mRNAs_p0.01.txt",sep="\t")

### Heatmap with mRNAs with p<0.01

In [ ]:
#heatmap
#subset miRNAs from lcpm normalized data
mRNAs<-as.list(final$Row.names)
lcpm.norm.heatmap<-as.matrix(subset(norm,rownames(norm) %in% mRNAs))

## Get some nicer colours
mypalette <- brewer.pal(11,"RdYlBu")
morecols <- colorRampPalette(mypalette)
# Set up colour vector for celltype variable
col.cell <- c("purple","orange")[group]
heatmap.2(lcpm.norm.heatmap,col=rev(morecols(50)),trace="none", main="p<0.01 TMM normalized",
          ColSideColors=col.cell,scale="row",margins=c(9,9), cexCol=0.8)

### Biomart to get gene information from ENSEMBL IDs

In [ ]:
ENSID<-final$Row.names
genes<-ENSID
mart <- useDataset("mmusculus_gene_ensembl", useMart("ensembl"))
attributes<-listAttributes(mart)
G_list <- data.table(getBM(filters= "ensembl_gene_id", attributes= c("ensembl_gene_id", 
        "description","uniprot_gn_symbol"),values=genes,mart= mart))
write.table(G_list,file="output/biomart_genes_list_edgeR.txt",sep="\t")

### Heatmap with top 50 dysregulated mRNAs with gene names

In [ ]:
top50<-as.matrix(read.delim("data/top50_edgeR.txt",header=TRUE,row.names=1))
mypalette <- brewer.pal(11,"RdYlBu")
morecols <- colorRampPalette(mypalette)
heatmap.2(top50,col=rev(morecols(50)),trace="none", main="Top 50 mRNAs (p<0.01)",scale="row",margins=c(9,9), 
          cexCol=1,cexRow=1)